In [53]:
import pandas as pd
import numpy as np

# Load the uploaded file to analyze its structure and content
data = pd.read_csv(r'C:\Users\Ane\Downloads\DAS-Homeworks-main\Домашна 1\stock_market.csv')

# Display the first few rows of the dataset to understand its structure
data.info()


C:\Users\Ane\AppData\Local\Temp\ipykernel_18668\176866895.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'C:\Users\Ane\Downloads\DAS-Homeworks-main\Домашна 1\stock_market.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299699 entries, 0 to 299698
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Date                        299699 non-null  object
 1   Last trade price            292699 non-null  object
 2   Max                         28357 non-null   object
 3   Min                         28357 non-null   object
 4   Avg. Price                  292699 non-null  object
 5   %chg.                       292688 non-null  object
 6   Volume                      299699 non-null  object
 7   Turnover in BEST in denars  299699 non-null  object
 8   Total turnover in denars    299699 non-null  object
 9   Issuer                      299699 non-null  object
dtypes: object(10)
memory usage: 22.9+ MB


In [69]:
unique_issuers = data['Issuer'].unique()

# Create a dictionary to store DataFrames for each issuer
issuer_dfs = {}

# Split the data for each issuer
for issuer in unique_issuers:
    issuer_dfs[issuer] = data[data['Issuer'] == issuer].copy()


In [73]:
issuer_dfs

{'SPOL':              Date  Last trade price  Max  Min Avg. Price %chg. Volume  \
 220708 2014-11-17               NaN  NaN  NaN        NaN   NaN      0   
 220707 2014-11-18               NaN  NaN  NaN        NaN   NaN      0   
 220706 2014-11-19               NaN  NaN  NaN        NaN   NaN      0   
 220705 2014-11-20               NaN  NaN  NaN        NaN   NaN      0   
 220704 2014-11-21               NaN  NaN  NaN        NaN   NaN      0   
 ...           ...               ...  ...  ...        ...   ...    ...   
 218255 2024-11-08               NaN  NaN  NaN        NaN   NaN      0   
 218254 2024-11-11               NaN  NaN  NaN        NaN   NaN      0   
 218253 2024-11-12               NaN  NaN  NaN        NaN   NaN      0   
 218252 2024-11-13               NaN  NaN  NaN        NaN   NaN      0   
 218251 2024-11-14               NaN  NaN  NaN        NaN   NaN      0   
 
        Turnover in BEST in denars Total turnover in denars Issuer  
 220708                          

In [54]:
# Data preprocessing
# Convert 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Remove rows with invalid dates
data = data.dropna(subset=['Date'])

# Sort by date in ascending order for time-series analysis
data = data.sort_values(by='Date')

# Remove commas from 'Last trade price' and convert to numeric
data['Last trade price'] = pd.to_numeric(data['Last trade price'].str.replace(',', ''), errors='coerce')

# Select necessary columns for technical analysis
technical_data = data[['Date', 'Last trade price']].dropna()

# Rename columns for easier handling
technical_data.rename(columns={'Last trade price': 'Price'}, inplace=True)

technical_data.head()


,Date,Price
13613,2014-11-17,307.0
6611,2014-11-17,12000.0
4915,2014-11-17,4900.0
228082,2014-11-17,440.0
278733,2014-11-17,3900.0


In [55]:
import ta

# Function to calculate technical indicators
def calculate_indicators(df):
    # Calculate RSI
    df['RSI_1D'] = ta.momentum.RSIIndicator(close=df['Price'], window=14).rsi()
    
    # Calculate Stochastic Oscillator (using High, Low, Close, but here using Price for simplicity)
    df['Stoch_1D'] = ta.momentum.StochRSIIndicator(close=df['Price'], window=14).stochrsi()
    
    # Calculate MACD (Moving Average Convergence Divergence)
    macd = ta.trend.MACD(close=df['Price'])
    df['MACD_1D'] = macd.macd_diff()
    
    # Calculate Williams %R
    df['Williams_%R_1D'] = ta.momentum.WilliamsRIndicator(
        high=df['Price'], low=df['Price'], close=df['Price'], lbp=14
    ).williams_r()
    
    # Calculate CCI (Commodity Channel Index)
    df['CCI_1D'] = ta.trend.CCIIndicator(
        high=df['Price'], low=df['Price'], close=df['Price'], window=14
    ).cci()
    weights = np.array(range(1, 15))
    weights = weights / weights.sum()
    
    df['SMA_1D'] = df['Price'].rolling(window=14).mean()
    df['EMA_1D'] = df['Price'].ewm(span=14, adjust=False).mean()
    
    # Use convolution for WMA calculation
    df['WMA_1D'] = df['Price'].rolling(window=14).apply(lambda x: np.dot(x, weights), raw=True)
    
    df['HMA_1D'] = (2 * df['EMA_1D'] - df['SMA_1D']).rolling(14).mean()
    df['TEMA_1D'] = 3 * df['EMA_1D'] - 3 * df['EMA_1D'].ewm(span=14).mean() + df['EMA_1D'].ewm(span=14).mean()
    

    return df

# Calculate indicators for the given dataset
technical_data = calculate_indicators(technical_data)

# Show a sample of the processed dataset
technical_data


,Date,Price,RSI_1D,Stoch_1D,MACD_1D,Williams_%R_1D,CCI_1D,SMA_1D,EMA_1D,WMA_1D,HMA_1D,TEMA_1D
13613,2014-11-17,307.0,NaN,NaN,NaN,NaN,NaN,NaN,307.000000,NaN,NaN,307.000000
6611,2014-11-17,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,1866.066667,NaN,NaN,3313.771429
4915,2014-11-17,4900.0,NaN,NaN,NaN,NaN,NaN,NaN,2270.591111,NaN,NaN,3665.257204
228082,2014-11-17,440.0,NaN,NaN,NaN,NaN,NaN,NaN,2026.512296,NaN,NaN,2655.694249
278733,2014-11-17,3900.0,NaN,NaN,NaN,NaN,NaN,NaN,2276.310657,NaN,NaN,3110.592702
...,...,...,...,...,...,...,...,...,...,...,...,...
119924,2024-11-14,89458.0,78.977360,1.000000,5437.600932,-0.000000,451.159856,9242.928571,15342.140746,15047.095238,5078.174942,34739.508937
176560,2024-11-14,315.0,47.814032,0.016257,2886.399693,-99.748232,-50.343473,9261.857143,13338.521980,13856.704762,5917.877466,26676.635218
69826,2024-11-14,1790.0,48.178401,0.026318,1212.455954,-98.097753,-42.823861,9379.000000,11798.719049,12860.457143,6629.196525,20689.425442
127267,2024-11-14,99.0,47.766636,0.013300,-28.699037,-99.989929,-52.259421,9363.571429,10238.756509,11623.123810,7191.116921,15240.100314


In [57]:
# Calculate RSI (Relative Strength Index)
def calculate_rsi(prices, window=14):
    delta = prices.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calculate SMA (Simple Moving Average)
def calculate_sma(prices, window=14):
    return prices.rolling(window=window).mean()

# Add RSI and SMA to the dataset
technical_data['RSI_1D'] = calculate_rsi(technical_data['Price'], window=14)
technical_data['SMA_1D'] = calculate_sma(technical_data['Price'], window=14)

# Display a sample of the dataset with RSI and SMA
technical_data


,Date,Price,RSI_1D,Stoch_1D,MACD_1D,Williams_%R_1D,CCI_1D,SMA_1D,EMA_1D,WMA_1D,HMA_1D,TEMA_1D
13613,2014-11-17,307.0,NaN,NaN,NaN,NaN,NaN,NaN,307.000000,NaN,NaN,307.000000
6611,2014-11-17,12000.0,100.000000,NaN,NaN,NaN,NaN,NaN,1866.066667,NaN,NaN,3313.771429
4915,2014-11-17,4900.0,62.219976,NaN,NaN,NaN,NaN,NaN,2270.591111,NaN,NaN,3665.257204
228082,2014-11-17,440.0,50.285985,NaN,NaN,NaN,NaN,NaN,2026.512296,NaN,NaN,2655.694249
278733,2014-11-17,3900.0,56.725190,NaN,NaN,NaN,NaN,NaN,2276.310657,NaN,NaN,3110.592702
...,...,...,...,...,...,...,...,...,...,...,...,...
119924,2024-11-14,89458.0,84.922613,1.000000,5437.600932,-0.000000,451.159856,9242.928571,15342.140746,15047.095238,5078.174942,34739.508937
176560,2024-11-14,315.0,50.061397,0.016257,2886.399693,-99.748232,-50.343473,9261.857143,13338.521980,13856.704762,5917.877466,26676.635218
69826,2024-11-14,1790.0,50.377564,0.026318,1212.455954,-98.097753,-42.823861,9379.000000,11798.719049,12860.457143,6629.196525,20689.425442
127267,2024-11-14,99.0,49.950619,0.013300,-28.699037,-99.989929,-52.259421,9363.571429,10238.756509,11623.123810,7191.116921,15240.100314


In [65]:
def generate_signals(df):
        # Buy/Sell signals for SMA
    df['SMA_Signal'] = np.where(df['SMA_1D'] > df['Price'], 'Buy',
                               np.where(df['SMA_1D'] < df['Price'], 'Sell', 'Hold'))
    
    # Buy/Sell signals for EMA
    df['EMA_Signal'] = np.where(df['EMA_1D'] > df['Price'], 'Buy',
                               np.where(df['EMA_1D'] < df['Price'], 'Sell', 'Hold'))
    
    # Buy/Sell signals for RSI
    df['RSI_Signal'] = np.where(df['RSI_1D'] < 30, 'Buy',
                               np.where(df['RSI_1D'] > 70, 'Sell', 'Hold'))
    
    # Buy/Sell signals for MACD
    df['MACD_Signal'] = np.where(df['MACD_1D'] > 0, 'Buy',
                                np.where(df['MACD_1D'] < 0, 'Sell', 'Hold'))
    
    # Additional signals based on Williams %R
    df['Williams_Signal'] = np.where(df['Williams_%R_1D'] < -80, 'Buy',
                                    np.where(df['Williams_%R_1D'] > -20, 'Sell', 'Hold'))
    
    # CCI signals
    df['CCI_Signal'] = np.where(df['CCI_1D'] < -100, 'Buy',
                               np.where(df['CCI_1D'] > 100, 'Sell', 'Hold'))
    
    # Count buy and sell signals for each row
    signal_columns = ['SMA_Signal', 'EMA_Signal', 'RSI_Signal', 
                     'MACD_Signal', 'Williams_Signal', 'CCI_Signal']
    
    df['Buy_Count'] = (df[signal_columns] == 'Buy').sum(axis=1)
    df['Sell_Count'] = (df[signal_columns] == 'Sell').sum(axis=1)
    
    # Generate composite signal based on majority
    df['Composite_Signal'] = np.where(df['Buy_Count'] > df['Sell_Count'], 'Buy',
                                     np.where(df['Sell_Count'] > df['Buy_Count'], 'Sell', 'Hold'))
    
    # Clean up temporary columns
    df = df.drop(['Buy_Count', 'Sell_Count'], axis=1)
    
    return df


generate_signals=generate_signals(technical_data)

In [67]:
generate_signals

,Date,Price,RSI_1D,Stoch_1D,MACD_1D,Williams_%R_1D,CCI_1D,SMA_1D,EMA_1D,WMA_1D,HMA_1D,TEMA_1D,SMA_Signal,EMA_Signal,RSI_Signal,MACD_Signal,Williams_Signal,CCI_Signal,Composite_Signal
13613,2014-11-17,307.0,NaN,NaN,NaN,NaN,NaN,NaN,307.000000,NaN,NaN,307.000000,Hold,Hold,Hold,Hold,Hold,Hold,Hold
6611,2014-11-17,12000.0,100.000000,NaN,NaN,NaN,NaN,NaN,1866.066667,NaN,NaN,3313.771429,Hold,Sell,Sell,Hold,Hold,Hold,Sell
4915,2014-11-17,4900.0,62.219976,NaN,NaN,NaN,NaN,NaN,2270.591111,NaN,NaN,3665.257204,Hold,Sell,Hold,Hold,Hold,Hold,Sell
228082,2014-11-17,440.0,50.285985,NaN,NaN,NaN,NaN,NaN,2026.512296,NaN,NaN,2655.694249,Hold,Buy,Hold,Hold,Hold,Hold,Buy
278733,2014-11-17,3900.0,56.725190,NaN,NaN,NaN,NaN,NaN,2276.310657,NaN,NaN,3110.592702,Hold,Sell,Hold,Hold,Hold,Hold,Sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119924,2024-11-14,89458.0,84.922613,1.000000,5437.600932,-0.000000,451.159856,9242.928571,15342.140746,15047.095238,5078.174942,34739.508937,Sell,Sell,Sell,Buy,Sell,Sell,Sell
176560,2024-11-14,315.0,50.061397,0.016257,2886.399693,-99.748232,-50.343473,9261.857143,13338.521980,13856.704762,5917.877466,26676.635218,Buy,Buy,Hold,Buy,Buy,Hold,Buy
69826,2024-11-14,1790.0,50.377564,0.026318,1212.455954,-98.097753,-42.823861,9379.000000,11798.719049,12860.457143,6629.196525,20689.425442,Buy,Buy,Hold,Buy,Buy,Hold,Buy
127267,2024-11-14,99.0,49.950619,0.013300,-28.699037,-99.989929,-52.259421,9363.571429,10238.756509,11623.123810,7191.116921,15240.100314,Buy,Buy,Hold,Sell,Buy,Hold,Buy
